In [14]:
import numpy as np
import PySimpleGUI as sg
import serial
import random as random
import time as t
from scipy.constants import h, c, k
from colour_system import *

sunriseDat = np.genfromtxt("rayleigh_corrected_rgb_array", delimiter = ',')
starData = np.genfromtxt('hardcoded_stars.dat')
starNames = ['Betelgeuse', 'Polaris', 'Proxima Centauri', 'Vega', 'S Cassiopeiae', 'Tatooine', 'Albireo']

temp = 5800
cs = cs_hdtv
lam = np.arange(380., 781., 5)

def send_data(k, R, G, B, brightness):
    infoString = '<' + str(k) + ',' + str(R) + ',' + str(G) + ',' + str (B) + ',' + str(brightness) + '>'
    infoByte = bytes(infoString, 'utf-8')
    ser.write(infoByte)

def rgb_to_hex(rgb):
    
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

def get_color_step(color1, color2, t):

    r = int(color1[0] + (color2[0] - color1[0]) * t)
    g = int(color1[1] + (color2[1] - color1[1]) * t)
    b = int(color1[2] + (color2[2] - color1[2]) * t)

    return (r, g, b)

def planck(lam, T):

    lam_m = lam / 1.e9
    fac = h*c/lam_m/k/T
    B = 2*h*c**2/lam_m**5 / (np.exp(fac) - 1)
    return B

def blackbody_color(T):
    spec = planck(lam, T)
    rgb = cs.spec_to_rgb(spec)
    return np.round(rgb*255)

def blackbody_rayleigh(T):
    spec = planck(lam, T)
    rgb = cs.spec_to_rgb(spec/lam**4)
    return np.round(rgb*255)

def smooth_transition(dataName):
    
    duration = 60
    i = 0
    
    if values['-INPUT-'] != '':
        duration = int(values['-INPUT-'])

    print(duration)
    steps = int(10*(duration/60))
    print(steps)

    while i < np.shape(dataName)[0]-1 and event != 'Exit':
        print(i)
        
        if i == 0:
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(dataName[i,:]))
            window['-BACKGROUND-'].update(value = str(dataName[i,0]) + ', ' + str(dataName[i, 1]) + ', ' + str(dataName[i, 2]))
            window.read(timeout = 1*(duration/60))
            for m in random.sample(range(0, 16), 16):
                send_data(m, Red, Green, Blue, 255)
                t.sleep(0.05)
                
        start = dataName[i, :]
        end = dataName[i+1, :]

        for j in range(steps+1):
            t_time = j/steps
            interColor = get_color_step(start, end, t_time)

            window['-BACKGROUND-'].update(background_color=rgb_to_hex(interColor)) 
            window['-BACKGROUND-'].update(value = str(interColor[0]) + ', ' + str(interColor[1]) + ', ' + str(interColor[2]))
            window.read(timeout = 1*(duration/60))
            for m in random.sample(range(0, 16), 16):
                send_data(m, Red, Green, Blue, 255)
                t.sleep(0.05)

        i += 1

    if i == np.shape(dataName)[0]-1:
        window['-BACKGROUND-'].update(value = 'Done!')
        
PORT_NAME = '/dev/ttyACM0'        # Windows (Change to the COM port your Arduino is connected to)

# Open the serial port
ser = serial.Serial(PORT_NAME, 9600, timeout=1)
ser.flush()


l1 = sg.Text('R, G, B', font =('Arial Bold', 30), background_color = 'black', expand_x=True, key='-BACKGROUND-', justification = 'center')
l2 = sg.Text('Input sequence duration (60s default)', font=('Arial Bold', 12), expand_x=True, justification='left')
l3 = sg.Input('', enable_events=True, key='-INPUT-', font=('Arial Bold', 12), expand_x=True, justification='left')
l4 = sg.Button('Sunrise demo sequence', key = '-SUNRISE-', font =('Arial Bold', 12))
l5 = sg.Button('Sunset demo sequence', key = '-SUNSET-', font =('Arial Bold', 12))
l6 = sg.Text('Input blackbody temperature (5800K default)', font=('Arial Bold', 12), expand_x=True, justification='left')
l7 = sg.Input('', enable_events=True, key='-TEMP-', font=('Arial Bold', 12), expand_x=True, justification='left')
l8 = sg.Button('Show blackbody color', key = '-BLACKBODY-', font = ('Arial Bold', 12))
l9 = sg.Button('Show blackbody color with Rayleigh correction', key = '-BLACKBODY_CORR-', font = ('Arial Bold', 12))
l10 = sg.Combo(starNames, expand_x = True, font = ('Arial Bold', 15), readonly=False, key='-STARS-')
l11 = sg.Button('Show sky color', key = '-STAR_COLOR-', font = ('Arial Bold', 15))
l12 = sg.Exit(font =('Arial Bold', 15))

layout = [[l1], [l2], [l3], [l4], [l5], [l6], [l7], [l8], [l9], [l10], [l11], [l12]]

window = sg.Window('SUNLAMP', layout, finalize=True)

while True:
        
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Exit':
        for j in random.sample(range(0, 16), 16):
            send_data(j, 0, 0, 0, 0) 
            t.sleep(0.01)
        ser.close()
        break
        
    if event == '-INPUT-':
        if len(values['-INPUT-']) > 1:
            if values['-INPUT-'][-1] not in ('0123456789'):
                sg.popup("Only digits allowed")
                window['-INPUT-'].update(values['-INPUT-'][:-1])
                
    if event == '-TEMP-':
        if len(values['-TEMP-']) > 1:
            if values['-TEMP-'][-1] not in ('0123456789'):
                sg.popup("Only digits allowed")
                window['-TEMP-'].update(values['-TEMP-'][:-1])
                
    if event == '-SUNRISE-':
        smooth_transition(sunriseDat)
        
    if event == '-SUNSET-':
        smooth_transition(np.flipud(sunriseDat))
        
    if event == '-BLACKBODY-':
        if values['-TEMP-'] != '':
            temp = int(values['-TEMP-'])
        tempColor = blackbody_color(temp)
        Red, Green, Blue = tempColor[0], tempColor[1], tempColor[2]
        window['-BACKGROUND-'].update(background_color=rgb_to_hex(tempColor)) 
        window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
        window.read(timeout = 1)
        for j in random.sample(range(0, 16), 16):
            send_data(j, Red, Green, Blue, 255)
            t.sleep(0.05)

   
    if event == '-BLACKBODY_CORR-':
        if values['-TEMP-'] != '':
            temp = int(values['-TEMP-'])
        tempColor2 = blackbody_rayleigh(temp)
        Red, Green, Blue = tempColor2[0], tempColor2[1], tempColor2[2]
        window['-BACKGROUND-'].update(background_color=rgb_to_hex(tempColor2)) 
        window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
        window.read(timeout = 1)
        for j in random.sample(range(0, 16), 16):
            send_data(j, Red, Green, Blue, 255) 
            t.sleep(0.05)
        
    if event == '-STAR_COLOR-':
        selected_star = values['-STARS-']
        
        if selected_star == 'Betelgeuse':
            Red, Green, Blue = starData[0,0], starData[0,1], starData[0,2]
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[0, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        
        elif selected_star == 'Polaris':
            Red, Green, Blue = starData[1,0], starData[1,1], starData[1,2]
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[1, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        
        elif selected_star == 'Proxima Centauri':
            Red, Green, Blue = starData[2,0], starData[2,1], starData[2,2]
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[2, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        
        elif selected_star == 'Vega':
            Red, Green, Blue = starData[3,0], starData[3,1], starData[3,2]
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[3, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        
        elif selected_star == 'S Cassiopeiae':
            Red, Green, Blue = starData[4,0], starData[4,1], starData[4,2]
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[4, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        
        elif selected_star == 'Tatooine':
            Red, Green, Blue = starData[5,0], starData[5,1], starData[5,2]
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[5, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        
        elif selected_star == 'Albeireo':
            Red, Green, Blue = starData[6,0], starData[6,1], starData[6,2] 
            window['-BACKGROUND-'].update(background_color=rgb_to_hex(starData[6, :])) 
            window['-BACKGROUND-'].update(value = str(Red) + ', ' + str(Green) + ', ' + str(Blue))
            window.read(timeout = 1)
            for j in random.sample(range(0, 16), 16):
                send_data(j, Red, Green, Blue, 255) 
                t.sleep(0.05)
        else:
            sg.popup("Please select a star")
        
window.close()
for j in random.sample(range(0, 16), 16):
    send_data(j, 0, 0, 0, 0) 
    t.sleep(0.01)
ser.close()

PortNotOpenError: Attempting to use a port that is not open